In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import json
import csv
import os
import random
from tempfile import gettempdir

import numpy as np
np.set_printoptions(threshold = np.nan)
import pandas as pd
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import shutil


# tf.enable_eager_execution()
tf.executing_eagerly()

False

## Read dataset

In [2]:
ls -la ../maniac/arc/ARC-V1-Feb2018-2/ARC-Easy

total 6680
drwxr-xr-x@ 8 ryangillard  eng     256 Jan 18 11:57 ./
drwxr-xr-x@ 7 ryangillard  eng     224 Jan 18 11:58 ../
-rwxr-xr-x@ 1 ryangillard  eng  159159 Mar  1  2018 ARC-Easy-Dev.csv*
-rwxr-xr-x@ 1 ryangillard  eng  214423 Mar  1  2018 ARC-Easy-Dev.jsonl*
-rwxr-xr-x@ 1 ryangillard  eng  666020 Mar  1  2018 ARC-Easy-Test.csv*
-rwxr-xr-x@ 1 ryangillard  eng  895104 Mar  1  2018 ARC-Easy-Test.jsonl*
-rwxr-xr-x@ 1 ryangillard  eng  629638 Mar  1  2018 ARC-Easy-Train.csv*
-rwxr-xr-x@ 1 ryangillard  eng  844113 Mar  1  2018 ARC-Easy-Train.jsonl*


In [ ]:
file_loc = '../maniac/arc/ARC-V1-Feb2018-2/ARC-Easy/ARC-Easy-Dev.jsonl'

In [ ]:
with open(file_loc) as f:
    content = f.readlines()
print(len(content))

In [ ]:
content[0]

In [ ]:
json_string = json.loads(content[0])

In [ ]:
json_string

### Weird entries
Some have numbered answers, fill in the blank

In [ ]:
json_string = json.loads(content[8])

In [ ]:
json_string

## Write jsonl data to csv as labeled dataset

In [11]:
import re
regex = r"(?<!\d)[.,;:?](?!\d)"

letter_answers = ['A', 'B', 'C', 'D', 'E']
number_answers = ['1', '2', '3', '4', '5']

def extract_question_answer(str_line):
  """Reads in line as string, returns tuple of question, answer"""
  json_line = json.loads(str_line)
  
  question = json_line['question']['stem']
  
  choices = [choice['text'] for choice in json_line['question']['choices']]
  
  answer = json_line['answerKey']
  if answer in letter_answers:
    answer_idx = letter_answers.index(answer)
  else: 
    answer_idx = number_answers.index(answer)
  answer_text = json_line['question']['choices'][answer_idx]['text']
  
  question_formatted = "<s> " + re.sub(
    regex, "", question, 0).replace("\"","").replace("\'","").replace(",","").replace(";","") + " </s>"
#   print("question_formatted = \n{}".format(question_formatted))
  choices_formatted = ";".join(
    ["<s> " + re.sub(
      regex, "", choice, 0).replace("\"","").replace("\'","").replace(",","").replace(";","") + " </s>" 
     for choice in choices])
#   print("choices_formatted = \n{}".format(choices_formatted))
  
  return question_formatted, choices_formatted, answer_idx

In [12]:
import io
import codecs

filename = "train_data.csv"
file_loc = "../maniac/arc/ARC-V1-Feb2018-2/ARC-Easy/ARC-Easy-Train.jsonl"

if os.path.exists(filename):
  os.remove(filename)
  
with io.open(file_loc, encoding='utf-8') as f:
  with open(filename, "w") as out:
    for line in f:
      question, choices, answer_idx = extract_question_answer(line)
      with codecs.open(filename, "a", "utf-8") as temp:
        temp.write(question + "," + choices + "," + str(answer_idx) + "\n")

In [13]:
import io
import codecs

filename = "eval_data.csv"
file_loc = "../maniac/arc/ARC-V1-Feb2018-2/ARC-Easy/ARC-Easy-Dev.jsonl"

if os.path.exists(filename):
  os.remove(filename)
  
with io.open(file_loc, encoding='utf-8') as f:
  with open(filename, "w") as out:
    for line in f:
      question, choices, answer_idx = extract_question_answer(line)
      with codecs.open(filename, "a", "utf-8") as temp:
        temp.write(question + "," + choices + "," + str(answer_idx) + "\n")

In [14]:
!head -5 train_data.csv

<s> Which factor will most likely cause a person to develop a fever </s>,<s> a leg muscle relaxing after exercise </s>;<s> a bacterial population in the bloodstream </s>;<s> several viral particles on the skin </s>;<s> carbohydrates being digested in the stomach </s>,1
<s> Lichens are symbiotic organisms made of green algae and fungi What do the green algae supply to the fungi in this symbiotic relationship </s>,<s> carbon dioxide </s>;<s> food </s>;<s> protection </s>;<s> water </s>,1
<s> When a switch is used in an electrical circuit the switch can </s>,<s> cause the charge to build </s>;<s> increase and decrease the voltage </s>;<s> cause the current to change direction </s>;<s> stop and start the flow of current </s>,3
<s> Which of the following is an example of an assistive device </s>,<s> contact lens </s>;<s> motorcycle </s>;<s> raincoat </s>;<s> coffee pot </s>,0
<s> Rocks are classified as igneous metamorphic or sedimentary according to </s>,<s> their color </s>;<s> their shap

In [15]:
!head -5 eval_data.csv

<s> Which technology was developed most recently </s>,<s> cellular telephone </s>;<s> television </s>;<s> refrigerator </s>;<s> airplane </s>,0
<s> A student hypothesizes that algae are producers Which question will best help the student determine if this is correct </s>,<s> Do algae consume other organisms </s>;<s> Which organisms consume algae </s>;<s> Do algae use sunlight to make food </s>;<s> Could an ecosystem survive without algae </s>,2
<s> Soccer players use their muscle systems to kick a ball into a goal What organ system coordinates the muscles </s>,<s> The nervous system </s>;<s> The endocrine system </s>;<s> The respiratory system </s>;<s> The circulatory system </s>,0
<s> Planets in the solar system are in constant motion What factor has the greatest effect on the orbits of the planets </s>,<s> the size of the planets </s>;<s> gravitational pull of the Sun </s>;<s> the composition of the planets </s>;<s> electromagnetic radiation from the Sun </s>,1
<s> How is a pond diff

## Learn embeddings from ARC corpus

In [16]:
!head -10 ARC_Corpus_small.txt

Large international companies are involved in bauxite, iron ore, diamond, and gold mining operations.
Paleoceanography, 8(2): 193-208.
Of course, for many in the media, “hydrogen sulphide delivery helps prevent disease damage in cells in certain disease models” will always be trumped by “farts cure cancer” when it comes to headlines.
The same problems apply with wolf-domestic dog hybrids.
taking stock of delightful days
The an- tlu-opologist and the ethnologist find in trop- ical America some of the most complicated and interesting problems of research.
ORDER ODONATA (Damselflies and Dragonflies) Diagnosis: large, to over three inches long; four wings, transparent and membranous, held vertically (damselflies) or laterally (dragonflies) at rest; chew- ing mouth parts, tooth-like; nymphs aquatic, feeding on mosquito larvae to small fish; adults terrestrial, feeding on other insects (Figure 14.27).
until they institute such safeguards and assurances of chaste maidenhood as characterize He

In [130]:
import re
regex = r"(?<!\d)[.,;:?`\"\'\(\)\[\]\{\}\\/“”](?!\d)"

# Step 1: Download the data.
filename = "ARC_Corpus.txt"

# Read the data into a list of strings.
def read_data(filename):
  with open(filename, "r") as ins:
    sentence_list = []
    for line in ins:
      line = line.replace('\n', '')
      line = "<s> " + re.sub(
        regex, "", line, 0).replace("("," ").replace(")"," ").replace("- ","").replace("“","").replace("”","").replace("\"","").replace("\'","").replace("  "," ") + " </s>"
      line = line.split(' ')
      line[:] = [word for word in line if (word != " " and word != "\n" and word != "")]
      sentence_list.append(line)
      
  vocabulary_list = [word for sentence in sentence_list for word in sentence]
    
  return vocabulary_list

vocabulary = read_data(filename)
print(len(vocabulary))

266778143


In [131]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 50000

def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [["UNK", -1]]

  count.extend(collections.Counter(words).most_common(n_words - 1))
  
  unique_words = [word_tuple[0] for word_tuple in count]
  
  def fix_start_stop_maybe(count, unique_words, token_string, token_index):
    index = unique_words.index(token_string) if token_string in unique_words else -1
    
    if index != token_index:
      count.insert(token_index, count.pop(index))
      unique_words.insert(token_index, unique_words.pop(index))
    return
    
  fix_start_stop_maybe(count, unique_words, "<s>", 1)
  fix_start_stop_maybe(count, unique_words, "</s>", 2)
  
  count[:] = count[0:vocabulary_size]
  
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
    
  data = list()
  unk_count = 0
  for word in words:
    index = dictionary.get(word, 0)
    if index == 0:  # dictionary["UNK"]
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary

# Filling 4 global variables:
# data - list of codes (integers from 0 to vocabulary_size-1).
#   This is the original text but words are replaced by their codes
# count - map of words(strings) to count of occurrences
# dictionary - map of words(strings) to their codes(integers)
# reverse_dictionary - maps codes(integers) to words(strings)
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary, vocabulary_size)
# del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 17625492], ('<s>', 14621860), ('</s>', 14621856), ('the', 11738977), ('of', 7788564)]
Sample data [1, 3022, 1747, 1582, 12, 807, 8, 48802, 1485, 8321] ['<s>', 'Large', 'international', 'companies', 'are', 'involved', 'in', 'bauxite', 'iron', 'ore']


In [132]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(
    shape = (batch_size), 
    dtype = np.int32)
  labels = np.ndarray(
    shape = (batch_size, 1), 
    dtype = np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  if data_index + span > len(data):
    data_index = 0
  buffer.extend(data[data_index:data_index + span])
  data_index += span
  for i in range(batch_size // num_skips):
    context_words = [w for w in range(span) if w != skip_window]
    words_to_use = random.sample(context_words, num_skips)
    for j, context_word in enumerate(words_to_use):
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[context_word]
    if data_index == len(data):
      buffer[:] = data[:span]
      data_index = span
    else:
      buffer.append(data[data_index])
      data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size = 8, num_skips = 2, skip_window = 1)
# This is first doing target word with left word, then target word with right word
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
    '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

3022 Large -> 1 <s>
3022 Large -> 1747 international
1747 international -> 3022 Large
1747 international -> 1582 companies
1582 companies -> 12 are
1582 companies -> 1747 international
12 are -> 807 involved
12 are -> 1582 companies


In [133]:
# Step 4: Build and train a skip-gram model.
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1     # How many words to consider left and right.
num_skips = 2     # How many times to reuse an input to generate a label.
num_sampled = 64    # Number of negative examples to sample.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. These 3 variables are used only for
# displaying model accuracy, they don't affect calculation.
valid_size = 16   # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
# Generate a uniform random sample from np.arange(valid_window) of size valid_size without replacement
valid_examples = np.random.choice(
  a = valid_window, 
  size = valid_size, 
  replace = False)

graph = tf.Graph()

with graph.as_default():
  # Input data.
  train_inputs = tf.placeholder(dtype = tf.int32, shape = [batch_size]) # batch_size vector
  train_labels = tf.placeholder(dtype = tf.int32, shape = [batch_size, 1]) # batch_size x 1 matrix
  valid_dataset = tf.constant(value = valid_examples, dtype = tf.int32) # valid_size vector
  #print("valid_dataset = ", valid_dataset)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device("/cpu:0"):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
      initial_value = tf.random_uniform(
        shape = [vocabulary_size, embedding_size], 
        minval = -1.0, 
        maxval = 1.0)) # embedding weights are vocabulary_size x embedding_size matrix
    #print("embeddings = ", embeddings)
    embed = tf.nn.embedding_lookup(
      params = embeddings, 
      ids = train_inputs) # embedding_size x embedding_size matrix
    #print("embed = ", embed)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
      initial_value = tf.truncated_normal(
        shape = [vocabulary_size, embedding_size], 
        stddev = 1.0 / math.sqrt(embedding_size))) # nce weights are vocabulary_size x embedding_size matrix
    #print("nce_weights = ", nce_weights)
    nce_biases = tf.Variable(
      initial_value = tf.zeros(
        shape = [vocabulary_size])) # nce biases are a vocabulary_size vector
    #print("nce_biases = ", nce_biases)

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  # Explanation of the meaning of NCE loss:
  #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
  loss = tf.reduce_mean(
    tf.nn.nce_loss(weights=nce_weights,
           biases=nce_biases,
           labels=train_labels,
           inputs=embed,
           num_sampled=num_sampled,
           num_classes=vocabulary_size)) # scalar
    #print("loss = ", loss)

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(
    x = tf.reduce_sum(
      input_tensor = tf.square(
        x = embeddings), 
      axis = 1, 
      keepdims = True)) # vocabulary_size x 1 matrix, summed along the embedding_size axis
  #print("norm = ", norm)
  normalized_embeddings = embeddings / norm # vocabulary_size x embedding_size matrix
  #print("normalized_embeddings = ", normalized_embeddings)
  valid_embeddings = tf.nn.embedding_lookup(
    params = normalized_embeddings, 
    ids = valid_dataset) # valid_size x embedding_size matrix
  #print("valid_embeddings = ", valid_embeddings)
  # cosine similarity
  similarity = tf.matmul(
    a = valid_embeddings, 
    b = normalized_embeddings, 
    transpose_b = True) #  valid_size x vocabulary_size = (valid_size x embedding_size matrix) x (vocabulary_size x embedding_size matrix)^T
  #print("similarity = ", similarity)

  # Add variable initializer.
  init = tf.global_variables_initializer()

In [134]:
# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph = graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = "Nearest to %s:" % valid_word # finding the k nearest neighbors to the validation word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)

  # Get final embeddings after training is complete
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  237.41680908203125
Nearest to your: ropes, Copulation, Cathedral, auxiliary, manmade, Mitochondria, pigment, chaparral,
Nearest to would: titanotheres, Have, parenteral, seawater, subduction, entertainers, IC, students’,
Nearest to at: coalition, Beginners, 41:, decipher, Prasad, thirst, Robbins, antecedent,
Nearest to between: permanence, Company, Scratch, tally, Magnificent, rejoice, monotypic, Werner,
Nearest to like: Mare, Peter, artefacts, 1959., secondhand, MATERIALS, carnivores, funeral,
Nearest to out: Café, HF, please, Peppers, backstory, flowered, ancient, certification,
Nearest to most: barometer, brokers, helper, Carrier, apartments, 42., Zen, bank,
Nearest to first: [134], extras, SUCH, coagulation, safely, waiting, -LRBdied, patrol,
Nearest to into: Survival, recount, Hyde, Underwood, adjective, ci, acquire, Bunker,
Nearest to when: Assad, spool, usefulness, dicots, tokens, depositing, trembling, Kittens,
Nearest to these: Adirondack

In [135]:
print(final_embeddings.shape) # from building the graph, embedding weights are vocabulary_size x embedding_size matrix

(50000, 128)


In [136]:
print("vocabulary_size:", vocabulary_size)
embedding_size = final_embeddings.shape[1]
print("embedding_size:", embedding_size)

vocabulary_size: 50000
embedding_size: 128


In [137]:
np.savetxt(
  fname = "vocab.tsv", 
  X = np.array(object = list(dictionary.keys())), 
  fmt = '%s', 
  delimiter = '\n') # write vocabulary to file
np.savetxt(
  fname = "word_embeddings.csv", 
  X = final_embeddings, 
  delimiter = ',')

In [138]:
!head -10 vocab.tsv

UNK
<s>
</s>
the
of
and
to
a
in
is


In [85]:
# !head -5 word_embeddings.csv

## Read in trained embeddings

In [2]:
pre_trained_embeddings_array = np.loadtxt(
  fname = "word_embeddings.csv", 
  dtype = np.float64, 
  delimiter = ',')

In [3]:
vocabulary_size = pre_trained_embeddings_array.shape[0]
embedding_size = pre_trained_embeddings_array.shape[1]
print("vocabulary_size = {} & embedding_size = {}".format(vocabulary_size, embedding_size))

vocabulary_size = 50000 & embedding_size = 128


## Create TensorFlow Dual LSTM Question Answering Model

In [4]:
UNKNOWN_TOKEN = 0
START_TOKEN = 1
STOP_TOKEN = 2

In [5]:
arguments = {}
# File arguments
arguments["train_file_pattern"] = "train_data.csv"
arguments["eval_file_pattern"] = "eval_data.csv"
arguments["output_dir"] = "trained_model"

# Sequence shape hyperparameters
arguments["batch_size"] = 16
arguments["reverse_feature_sequence"] = True

# LSTM hyperparameters
arguments["question_lstm_hidden_units"] = [256, 128, 64]
arguments["question_lstm_dropout_output_keep_probs"] = [0.99, 0.97, 0.95]
arguments["choices_lstm_hidden_units"] = [256, 128, 64]
arguments["choices_lstm_dropout_output_keep_probs"] = [0.99, 0.97, 0.95]

# DNN hyperparameters
arguments["use_question_dnn"] = True
arguments["question_dnn_hidden_units"] = [64, 32, 16]
arguments["question_dnn_dropout_rates"] = [0.01, 0.03, 0.05]
arguments["use_choices_dnn"] = True
arguments["choices_dnn_hidden_units"] = [64, 32, 16]
arguments["choices_dnn_dropout_rates"] = [0.01, 0.03, 0.05]
arguments["final_logits_size"] = 16

# Training parameters
arguments["train_steps"] = 1000
arguments["learning_rate"] = 0.1
arguments["start_delay_secs"] = 30
arguments["throttle_secs"] = 30

In [6]:
# Set logging to be level of INFO
tf.logging.set_verbosity(tf.logging.INFO)

In [7]:
# Determine CSV and label columns
CSV_COLUMNS = "question,choices,answer_idx".split(',')
LABEL_COLUMN = "answer_idx"
VOCAB_FILE_PATH = "vocab.tsv" # where vocabulary is saved, dynamically set in train_and_eval function
PADWORD = 'ZYXW'

# Set default values for each CSV column
DEFAULTS = [[""], [""], [0]]

In [8]:
# Create an input function reading a file using the Dataset API
# Then provide the results to the Estimator API
def read_dataset(filename, mode, batch_size, params):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.decode_csv(
        records = value_column, record_defaults = DEFAULTS, field_delim = ',')
      features = dict(zip(CSV_COLUMNS, columns))
      labels = tf.cast(x = features.pop(LABEL_COLUMN), dtype = tf.int64)
      
      return features, labels
    
    # Create list of files that match pattern
    file_list = tf.gfile.Glob(filename = filename)

    # Create dataset from file list
    dataset = tf.data.TextLineDataset(filenames = file_list)  # Read text file

    # Decode the CSV file into a features dictionary of tensors
    dataset = dataset.map(map_func = decode_csv)
    
    # Determine amount of times to repeat file based on if we are training or evaluating
    if mode == tf.estimator.ModeKeys.TRAIN:
      num_epochs = None # indefinitely
    else:
      num_epochs = 1 # end-of-input after this

    # Repeat files num_epoch times
    dataset = dataset.repeat(count = num_epochs)

    # Group the data into batches
    dataset = dataset.batch(batch_size = batch_size)
    
    # Determine if we should shuffle based on if we are training or evaluating
    if mode == tf.estimator.ModeKeys.TRAIN:
      dataset = dataset.shuffle(buffer_size = 10 * batch_size)

    # Create a iterator and then pull the next batch of features from the example queue
    batch_features, batch_labels = dataset.make_one_shot_iterator().get_next()

    return batch_features, batch_labels
  return _input_fn

In [18]:
# Create our model function to be used in our custom estimator
def dual_lstm_text_question_answer(features, labels, mode, params):
  # Function to split string tensors into multiple substring tensors
  def split_strings(features):
    # Function to split string tensors into substring tensors
    def split_string_into_substrings(string_tensor, delimiter):
      # Split string tensor into a sparse tensor based on delimiter
      # indices: shape = (cur_batch_size, 2), values: shape = (cur_batch_size,), dense_shape: shape = (2,)
      split_string_sparse_tensor = tf.string_split(
        source = string_tensor, 
        delimiter = delimiter)

      # Create a dense tensor of the float values that were converted from text csv
      # shape = (cur_batch_size, max_substrings_across_batch)
      split_string_dense_tensor = tf.sparse_tensor_to_dense(
        sp_input = split_string_sparse_tensor, 
        default_value = PADWORD)

      # The index for the sequence I am currently on, the first column of the sparse tensor indicies
      # shape = (cur_batch_size,)
      sequence_index = split_string_sparse_tensor.indices[:, 0]

      # The index for the timestep I am currently on, the second column of the sparse tensor indicies
      # shape = (cur_batch_size,)
      time_index = split_string_sparse_tensor.indices[:, 1]

      # The sequence lengths for each sequence
      # shape = (cur_batch_size,)
      lengths_vector = tf.segment_max(
        data = time_index, 
        segment_ids = sequence_index) + 1

      return split_string_dense_tensor, lengths_vector

    # Function to split a single string of multiple setences into words
    def split_multi_sentence_string_into_words(multi_sentence_string_tensor):
      # split_multi_sentence_string_dense_tensor.shape = (cur_batch_size, max_num_choices_across_batch)
      # num_sentences_vector.shape = (cur_batch_size,)
      split_multi_sentence_string_dense_tensor, num_sentences_vector = \
        split_string_into_substrings(string_tensor = multi_sentence_string_tensor, delimiter = ';')

      cur_batch_size = tf.shape(
        input = split_multi_sentence_string_dense_tensor, 
        out_type = tf.int64)[0] # shape = ()

      # Calculate the max number of choices across all questions in the batch
      max_num_choices_across_batch = tf.reduce_max(
        input_tensor = num_sentences_vector) # shape = ()

      split_multi_sentence_string_dense_tensor_flattened = tf.reshape(
        tensor = split_multi_sentence_string_dense_tensor, 
        shape = [cur_batch_size * max_num_choices_across_batch])

      # split_multi_sentence_word_string_dense_tensor_flattened.shape = (cur_batch_size * max_num_choices_across_batch, max_num_words_across_batch)
      # num_multi_sentence_word_tensor_flattened.shape = (cur_batch_size * max_num_choices_across_batch,)
      split_multi_sentence_word_string_dense_tensor_flattened, num_multi_sentence_word_tensor_flattened = \
        split_string_into_substrings(string_tensor = split_multi_sentence_string_dense_tensor_flattened, delimiter = ' ')

      max_num_words_across_batch = tf.reduce_max(
        input_tensor = num_multi_sentence_word_tensor_flattened) # shape = ()

      split_multi_sentence_word_string_dense_tensor = tf.reshape(
        tensor = split_multi_sentence_word_string_dense_tensor_flattened, 
        shape = [cur_batch_size, max_num_choices_across_batch, max_num_words_across_batch])

      num_multi_sentence_word_tensor = tf.reshape(
        tensor = num_multi_sentence_word_tensor_flattened, 
        shape = [cur_batch_size, max_num_choices_across_batch])

      return split_multi_sentence_word_string_dense_tensor, num_multi_sentence_word_tensor, split_multi_sentence_string_dense_tensor, num_sentences_vector

    # Start split_strings function
    # question_split_words_strings_tensor.shape = (cur_batch_size, max_num_question_words_across_batch)
    # question_num_words.shape = (cur_batch_size,)
    question_split_words_strings_tensor, question_num_words = \
      split_string_into_substrings(string_tensor = features["question"], delimiter = ' ')

    # choices_split_words_strings_tensor.shape = (cur_batch_size, max_num_choices_across_batch, max_num_choice_words_across_batch)
    # choices_num_words.shape = (cur_batch_size, max_num_choices)
    # choices_split_sentences_strings_tensor.shape = (cur_batch_size, max_num_choices)
    # choices_num_sentences. shape = (cur_batch_size,)
    choices_split_words_strings_tensor, choices_num_words, choices_split_sentences_strings_tensor, choices_num_sentences = \
      split_multi_sentence_string_into_words(features["choices"])

    return question_split_words_strings_tensor, question_num_words, choices_split_words_strings_tensor, choices_num_words, choices_split_sentences_strings_tensor, choices_num_sentences

  # question_split_words_strings_tensor.shape = (cur_batch_size, max_num_question_words_across_batch)
  # question_num_words.shape = (cur_batch_size,)
  # choices_split_words_strings_tensor.shape = (cur_batch_size, max_num_choices_across_batch, max_num_choice_words_across_batch)
  # choices_num_words.shape = (cur_batch_size, max_num_choices)
  # choices_split_sentences_strings_tensor.shape = (cur_batch_size, max_num_choices)
  # choices_num_sentences.shape = (cur_batch_size,)
  question_split_words_strings_tensor, question_num_words, choices_split_words_strings_tensor, choices_num_words, choices_split_sentences_strings_tensor, choices_num_sentences = \
    split_strings(features)
  
  # Map each word to respective integer index
  word_to_id_lookup_table = tf.contrib.lookup.index_table_from_file(
    vocabulary_file = "vocab.tsv",
    num_oov_buckets = 0,
    vocab_size = vocabulary_size,
    default_value = 0,  # for words not in vocabulary (OOV)
    delimiter = ' ')

  # shape = (cur_batch_size, max_num_question_words_across_batch)
  question_split_words_ids_tensor = word_to_id_lookup_table.lookup(
    question_split_words_strings_tensor)
  # shape = (cur_batch_size, max_num_choices_across_batch, max_num_choice_words_across_batch)
  choices_split_words_ids_tensor = word_to_id_lookup_table.lookup(
    choices_split_words_strings_tensor)
    
  # Load trained embeddings into variable
  # shape = (vocabulary_size, embedding_size)
  embeddings_placeholder = tf.placeholder(
    dtype = tf.float64, 
    shape = [vocabulary_size, embedding_size], 
    name = "embedding_placeholder")
  
  # shape = (vocabulary_size, embedding_size)
  embeddings_variable = tf.Variable(
    initial_value = embeddings_placeholder, 
    trainable = True, 
    name = "embedding_variable", 
    dtype = tf.float64, 
    expected_shape = [vocabulary_size, embedding_size])

  # Get dynamic batch size in case there was a partially filled batch
  cur_batch_size32 = tf.shape(
    input = question_split_words_ids_tensor, 
    out_type = tf.int32)[0] # shape = ()
  
  cur_batch_size64 = tf.shape(
    input = question_split_words_ids_tensor, 
    out_type = tf.int64)[0] # shape = ()
  
  # Calculate the max number of choices across all questions in the batch
  max_num_choices_across_batch = tf.reduce_max(
    input_tensor = choices_num_sentences) # shape = ()
  
  max_num_choice_words_across_batch = tf.reduce_max(
    input_tensor = choices_num_words) # shape = ()

  # Gather the embedding vectors for the question's words
  # shape = (cur_batch_size, max_num_question_words_across_batch, embedding_size)
  question_split_words_embeddings_tensor = tf.nn.embedding_lookup(
    params = embeddings_variable, 
    ids = question_split_words_ids_tensor)
  
  # Gather the embedding vectors for the answer's words
  # shape = (cur_batch_size, max_num_choices_across_batch, max_num_choice_words_across_batch, embedding_size)
  choices_split_words_embeddings_tensor = tf.nn.embedding_lookup(
    params = embeddings_variable, 
    ids = choices_split_words_ids_tensor)
  
  # It usually works better to have the features in reverse order due to higher correlation between the later feature timesteps and earlier label timesteps
  if params["reverse_feature_sequence"] == True:
    # shape = (cur_batch_size, max_num_question_words_across_batch, embedding_size)
    question_split_words_embeddings_tensor = tf.reverse_sequence(
      input = question_split_words_embeddings_tensor, 
      seq_lengths = question_num_words, 
      seq_axis = 1, 
      batch_axis = 0)

  ################################################################################
  
  # Function to create stack of LSTM
  def create_LSTM_stack(lstm_hidden_units, lstm_dropout_output_keep_probs):
    # First create a list of LSTM cells using our list of lstm hidden unit sizes
    lstm_cells = [tf.nn.rnn_cell.BasicLSTMCell(
      num_units = units, 
      state_is_tuple = True) 
                  for units in lstm_hidden_units] # list of LSTM cells

    # Next apply a dropout wrapper to our stack of LSTM cells, in this case just on the outputs
    dropout_lstm_cells = [tf.nn.rnn_cell.DropoutWrapper(
      cell = lstm_cells[cell_index], 
      input_keep_prob = 1.0, 
      output_keep_prob = lstm_dropout_output_keep_probs[cell_index], 
      state_keep_prob = 1.0) 
                          for cell_index in range(len(lstm_cells))]

    # Create a stack of layers of LSTM cells
    stacked_lstm_cells = tf.nn.rnn_cell.MultiRNNCell(
      cells = dropout_lstm_cells, 
      state_is_tuple = True) # combines list into MultiRNNCell object

    return stacked_lstm_cells
  
  ################################################################################

  # Create our question LSTM stack now
  question_stacked_lstm_cells = create_LSTM_stack(
    lstm_hidden_units = params["question_lstm_hidden_units"], 
    lstm_dropout_output_keep_probs = params["question_lstm_dropout_output_keep_probs"])

  # Encode the question embedding sequence using our question stack of LSTMs
  # question_outputs.shape = (cur_batch_size, max_question_num_words, question_lstm_hidden_units[-1])
  # question_states tuple of question final c_state and h_state for each layer
  question_outputs, question_states = tf.nn.dynamic_rnn(
    cell = question_stacked_lstm_cells, 
    inputs = question_split_words_embeddings_tensor,
    sequence_length = question_num_words,
    initial_state = question_stacked_lstm_cells.zero_state(
      batch_size = cur_batch_size32, 
      dtype = tf.float64), 
    dtype = tf.float64)

  # Extract the question final output
  # shape = (cur_batch_size, question_lstm_hidden_units[-1])
  question_final_outputs = tf.map_fn(
    fn = lambda x: question_outputs[x[0], x[1], :], 
    elems = (tf.range(start = 0, limit = cur_batch_size64, dtype = tf.int64), 
             question_num_words - 1), 
    dtype = tf.float64)
  
  ################################################################################
  
  # Create the input layer to our question DNN
  # shape = (cur_batch_size, question_lstm_hidden_units[-1])
  question_dnn_network = question_final_outputs
  
  if params["use_question_dnn"] == True:
    # Add hidden layers with the given number of units/neurons per question DNN layer
    for i in range(0, len(params['question_dnn_hidden_units'])):
      # shape = (cur_batch_size, question_dnn_hidden_units[i])
      question_dnn_network = tf.layers.dense(
        inputs = question_dnn_network, 
        units = params['question_dnn_hidden_units'][i], 
        activation = tf.nn.relu)
      question_dnn_network = tf.layers.dropout(
        inputs = question_dnn_network, 
        rate = params['question_dnn_dropout_rates'][i])
  
  # Add question final logits layer
  # shape = (cur_batch_size, final_logits_size)
  question_logits = tf.layers.dense(
    inputs = question_dnn_network, 
    units = params["final_logits_size"], 
    activation = None)

  ################################################################################
  
   # Create our choices LSTM stack now
  choices_stacked_lstm_cells = create_LSTM_stack(
    lstm_hidden_units = params["choices_lstm_hidden_units"], 
    lstm_dropout_output_keep_probs = params["choices_lstm_dropout_output_keep_probs"])
  
  # Encode the choices embedding sequence using our choices stack of LSTMs in while loop
  def while_loop_condition(choice_index, max_num_choices_across_batch, choices_split_words_embeddings_tensor, choices_num_words, cur_batch_size32, choices_outputs):
    # shape = ()
    condition = tf.less(
      x = choice_index, 
      y = max_num_choices_across_batch)

    return condition

  def while_loop_body(choice_index, max_num_choices_across_batch, choices_split_words_embeddings_tensor, choices_num_words, cur_batch_size32, choices_outputs):
    with tf.variable_scope(
      name_or_scope = "choices_lstm_stack", reuse = tf.AUTO_REUSE):
      # Run each choice through the choices LSTM stack
      # shape = (cur_batch_size, max_num_choices_words_across_batch, choices_lstm_hidden_units[-1])
      choice_outputs, _ = tf.nn.dynamic_rnn(
        cell = choices_stacked_lstm_cells, 
        inputs = choices_split_words_embeddings_tensor[:, choice_index, :, :],
        sequence_length = choices_num_words[:, choice_index],
        initial_state = choices_stacked_lstm_cells.zero_state(
          batch_size = cur_batch_size32, 
          dtype = tf.float64), 
        dtype = tf.float64)

    # Concatenate the outputs into choice collection
    # shape = (cur_batch_size, choice_index + 1, max_num_choices_words_across_batch, choices_lstm_hidden_units[-1])
    choices_outputs = tf.concat(
      values = [choices_outputs, tf.expand_dims(input = choice_outputs, axis = 1)], 
      axis = 1)

    # Increment to the next choice
    choice_index += 1
    
    return choice_index, max_num_choices_across_batch, choices_split_words_embeddings_tensor, choices_num_words, cur_batch_size32, choices_outputs

  # Initial loop variable and output tensor
  choice_index = tf.constant(value = 0, dtype = tf.int64) # shape = ()
  # shape = (cur_batch_size, 0, max_num_choices_words_across_batch, choices_lstm_hidden_units[-1])
  choices_outputs = tf.zeros(
    shape = [tf.cast(
      x = cur_batch_size32, 
      dtype = tf.int64), 
             0, 
             max_num_choice_words_across_batch, 
             params["choices_lstm_hidden_units"][-1]], 
    dtype = tf.float64)

  # Call the while loop
  # choices_outputs.shape = (cur_batch_size, max_num_choices_across_batch, max_num_choices_words_across_batch, choices_lstm_hidden_units[-1])
  choice_index, max_num_choices_across_batch, choices_split_words_embeddings_tensor, choices_num_words, cur_batch_size32, choices_outputs = \
    tf.while_loop(
      cond = while_loop_condition, 
      body = while_loop_body, 
      loop_vars = [choice_index, max_num_choices_across_batch, choices_split_words_embeddings_tensor, choices_num_words, cur_batch_size32, choices_outputs],
      shape_invariants = [
        choice_index.get_shape(), 
        max_num_choices_across_batch.get_shape(),
        choices_split_words_embeddings_tensor.get_shape(), 
        choices_num_words.get_shape(), 
        cur_batch_size32.get_shape(), 
        tf.TensorShape(dims = [None, None, None, params["choices_lstm_hidden_units"][-1]])
      ])

  # Extract the choices final output
  # shape = (cur_batch_size, max_num_choices_across_batch)
  tiled_batch_indices = tf.map_fn(
    fn = lambda x: tf.tile(
      input = [x], 
      multiples = [max_num_choices_across_batch]), 
    elems = tf.range(start = 0, limit = cur_batch_size64, dtype = tf.int64), 
    dtype = tf.int64)
  # shape = (cur_batch_size * max_num_choices_across_batch)
  tiled_batch_indices_flattened = tf.reshape(
    tensor = tiled_batch_indices, 
    shape = [cur_batch_size64 * max_num_choices_across_batch])
  # shape = (cur_batch_size * max_num_choices_across_batch)
  tiled_choice_indices = tf.tile(
    input = tf.range(start = 0, limit = max_num_choices_across_batch, dtype = tf.int64), 
    multiples = [cur_batch_size64])
  # shape = (cur_batch_size * max_num_choices_across_batch)
  choices_num_words_flattened = tf.reshape(
    tensor = choices_num_words, 
    shape = [cur_batch_size64 * max_num_choices_across_batch])

  # shape = (cur_batch_size * max_num_choices_across_batch, choices_lstm_hidden_units[-1])
  choices_final_outputs_flat = tf.map_fn(
    fn = lambda x: choices_outputs[x[0], x[1], x[2], :], 
    elems = (tiled_batch_indices_flattened, tiled_choice_indices, choices_num_words_flattened - 1), 
    dtype = tf.float64)

  ################################################################################

  # Create the input layer to our choices DNN
  # shape = (cur_batch_size * max_num_choices_across_batch, choices_lstm_hidden_units[-1])
  choices_dnn_network = choices_final_outputs_flat
  
  if params["use_choices_dnn"] == True:
    # Add hidden layers with the given number of units/neurons per question DNN layer
    for i in range(0, len(params['choices_dnn_hidden_units'])):
      # shape = (cur_batch_size * max_num_choices_across_batch, choices_dnn_hidden_units[i])
      choices_dnn_network = tf.layers.dense(
        inputs = choices_dnn_network, 
        units = params['choices_dnn_hidden_units'][i], 
        activation = tf.nn.relu)
      choices_dnn_network = tf.layers.dropout(
        inputs = choices_dnn_network, 
        rate = params['choices_dnn_dropout_rates'][i])
  
  # Add choices final logits layer
  # shape = (cur_batch_size * max_num_choices_across_batch, final_logits_size)
  choices_logits_flat = tf.layers.dense(
    inputs = choices_dnn_network, 
    units = params["final_logits_size"], 
    activation = None)

  # shape = (cur_batch_size, max_num_choices_across_batch, final_logits_size)
  choices_logits = tf.reshape(
    tensor = choices_logits_flat, 
    shape = [cur_batch_size64, max_num_choices_across_batch, params["final_logits_size"]])
  
  ################################################################################
  
  # Loss function, training/eval ops
  if mode == tf.estimator.ModeKeys.TRAIN:
    # shape = (cur_batch_size, final_logits_size)
    answer_logits = tf.map_fn(
      fn = lambda x: choices_logits[x[0], x[1], :], 
      elems = (tf.range(start = 0, limit = cur_batch_size64, dtype = tf.int64), labels), 
      dtype = tf.float64)
    
    predictions_dict = None
    loss = tf.losses.mean_squared_error(
      labels = answer_logits, predictions = question_logits)
    train_op = tf.contrib.layers.optimize_loss(
      loss = loss,
      global_step = tf.train.get_global_step(),
      learning_rate = params['learning_rate'],
      optimizer = "Adam")
    eval_metric_ops = None
    export_outputs = None
  else:
    # Normalize logits
    # shape = (cur_batch_size, final_logits_size)
    question_logits_normalized = tf.nn.l2_normalize(
      x = question_logits, axis = 1)
    # shape = (cur_batch_size, max_num_choices_across_batch, final_logits_size)
    choices_logits_normalized = tf.nn.l2_normalize(
      choices_logits, axis = 2)

    # Find cosine similiarites
    # shape = (cur_batch_size, max_num_choices_across_batch)
    cosine_similarities = tf.transpose(
      a = tf.map_fn(
        fn = lambda x: tf.reduce_sum(
          input_tensor = tf.multiply(
            x = question_logits_normalized, 
            y = choices_logits_normalized[:, x, :]), 
          axis = 1), 
        elems = tf.range(start = 0, limit = max_num_choices_across_batch, dtype = tf.int64), 
        dtype = tf.float64))
    
    # Find euclidean distances
    # shape = (cur_batch_size, max_num_choices_across_batch)
    euclidean_distances = tf.transpose(
      a = tf.map_fn(
        fn = lambda x: tf.norm(
          tensor = question_logits - choices_logits[:, x, :], 
          axis = 1), 
        elems = tf.range(start = 0, limit = max_num_choices_across_batch, dtype = tf.int64), 
        dtype = tf.float64))

#     # Choose the index with the highest cosine similarity for each element in the batch
#     predicted_answer_index = tf.argmax(input = cosine_similarities, axis = 1) # shape = (cur_batch_size,)

    # Choose the index with the lowest euclidean distance for each element in the batch
    # shape = (cur_batch_size,)
    predicted_answer_index = tf.map_fn(
      fn = lambda x: tf.argmin(
        input = euclidean_distances[x[0], 0:x[1]], 
        axis = 0, 
        output_type = tf.int64), 
      elems = (tf.range(start = 0, limit = cur_batch_size64, dtype = tf.int64), 
               choices_num_sentences), 
      dtype = tf.int64)
    
    if mode == tf.estimator.ModeKeys.EVAL:
      # shape = (cur_batch_size, final_logits_size)
      answer_logits = tf.map_fn(
        fn = lambda x: choices_logits[x[0], x[1], :], 
        elems = (tf.range(start = 0, limit = cur_batch_size64, dtype = tf.int64), labels), 
        dtype = tf.float64)
      
      predictions_dict = None
      loss = tf.losses.mean_squared_error(
        labels = answer_logits, predictions = question_logits)
      train_op = None
      eval_metric_ops = {
        "rmse": tf.metrics.root_mean_squared_error(
          labels = answer_logits, predictions = question_logits),
        "accuracy": tf.metrics.accuracy(
          labels = labels, predictions = predicted_answer_index),
        "mean_cosine_distance": tf.metrics.mean_cosine_distance(
          labels = tf.nn.l2_normalize(x = answer_logits, axis = 1), 
          predictions = tf.nn.l2_normalize(x = question_logits, axis = 1), 
          dim = 1)
      }
      export_outputs = None
    else:
      # shape = (cur_batch_size)
      predicted_answer_text = tf.gather_nd(
        params = choices_split_sentences_strings_tensor, 
        indices = tf.stack(
          values = [tf.range(start = 0, limit = cur_batch_size64, dtype = tf.int64), 
                    predicted_answer_index], 
          axis = 1))
      
      # Create predictions
      predictions_dict = {
        "cosine_similarities": cosine_similarities, 
        "euclidean_distances": euclidean_distances, 
        "predicted_answer_index": predicted_answer_index, 
        "question_text": features["question"],
        "predicted_answer_text": predicted_answer_text,
        "question_logits": question_logits,
        "choices_logits": choices_logits,
        "choices_num_sentences": choices_num_sentences,
        "choices_num_words": choices_num_words,
        "features['choices']": features["choices"],
        "max_num_choices_across_batch": tf.tile(
          input = [max_num_choices_across_batch], multiples = [cur_batch_size64]),
        "max_num_choice_words_across_batch": tf.tile(
          input = [max_num_choice_words_across_batch], multiples = [cur_batch_size64]),
        "choices_split_words_strings_tensor": choices_split_words_strings_tensor,
        "choices_split_sentences_strings_tensor": choices_split_sentences_strings_tensor,
        "choices_final_outputs_flat": tf.reshape(
          tensor = choices_final_outputs_flat, 
          shape = [cur_batch_size64, max_num_choices_across_batch, params["choices_lstm_hidden_units"][-1]]), 
        "choices_split_words_embeddings_tensor": choices_split_words_embeddings_tensor, 
        "choices_split_words_ids_tensor": choices_split_words_ids_tensor, 
        "choices_outputs": choices_outputs}
      loss = None
      train_op = None
      eval_metric_ops = None

      # Create export outputs
      export_outputs = {
        "predict_export_outputs": tf.estimator.export.PredictOutput(
          outputs = predictions_dict)}

  # Return EstimatorSpec
  return tf.estimator.EstimatorSpec(
  mode = mode,
  predictions = predictions_dict,
  loss = loss,
  train_op = train_op,
  eval_metric_ops = eval_metric_ops,
  export_outputs = export_outputs,
  scaffold = tf.train.Scaffold(
    init_feed_dict = {
      embeddings_placeholder: pre_trained_embeddings_array}))

In [19]:
# Create our serving input function to accept the data at serving and send it in the right format to our custom estimator
def serving_input_fn():
  # Create placeholders to accept the data sent to the model at serving time
  feature_placeholders = {
    # All features come in as a batch of strings, shape = (batch_size,)
    # This was so because of passing the arrays to online ml-engine prediction
    feature: tf.placeholder(
      dtype = tf.string, 
      shape = [None]) 
    for feature in CSV_COLUMNS[0:-1]
  }

  # Create feature tensors
  features = feature_placeholders

  return tf.estimator.export.ServingInputReceiver(
    features = features, 
    receiver_tensors = feature_placeholders)

In [20]:
# Create estimator to train and evaluate
def train_and_evaluate(args):
  # Modify run config
  run_config = tf.estimator.RunConfig(
    save_checkpoints_steps = 100,
    keep_checkpoint_max = 10)
  
  # Create our custom estimator using our model function
  estimator = tf.estimator.Estimator(
    model_fn = dual_lstm_text_question_answer,
    model_dir = args["output_dir"],
    config = run_config,
    params = {
      "batch_size": args["batch_size"],
      "reverse_feature_sequence": args["reverse_feature_sequence"],
      
      "question_lstm_hidden_units": args["question_lstm_hidden_units"],
      "question_lstm_dropout_output_keep_probs": args["question_lstm_dropout_output_keep_probs"],
      "choices_lstm_hidden_units": args["choices_lstm_hidden_units"],
      "choices_lstm_dropout_output_keep_probs": args["choices_lstm_dropout_output_keep_probs"],

      "use_question_dnn": args["use_question_dnn"],
      "question_dnn_hidden_units": args["question_dnn_hidden_units"],
      "question_dnn_dropout_rates": args["question_dnn_dropout_rates"],
      "use_choices_dnn": args["use_choices_dnn"],
      "choices_dnn_hidden_units": args["choices_dnn_hidden_units"],
      "choices_dnn_dropout_rates": args["choices_dnn_dropout_rates"],
      "final_logits_size": args["final_logits_size"], 

      "learning_rate": args["learning_rate"]})
  
  # Create train spec to read in our training data
  train_spec = tf.estimator.TrainSpec(
    input_fn = read_dataset(
      filename = args["train_file_pattern"], 
      mode = tf.estimator.ModeKeys.TRAIN, 
      batch_size = args["batch_size"],
      params = args),
    max_steps = args["train_steps"])
  
  # Create exporter to save out the complete model to disk
  exporter = tf.estimator.LatestExporter(
    name = "exporter", 
    serving_input_receiver_fn = serving_input_fn)
  
  # Create eval spec to read in our validation data and export our model
  eval_spec = tf.estimator.EvalSpec(
    input_fn = read_dataset(
      filename = args["eval_file_pattern"], 
      mode = tf.estimator.ModeKeys.EVAL, 
      batch_size = args["batch_size"],
      params = args),
    steps = None,
    start_delay_secs = args["start_delay_secs"], # start evaluating after N seconds
    throttle_secs = args["throttle_secs"],  # evaluate every N seconds
    exporters = exporter)
  
  # Create train and evaluate loop to train and evaluate our estimator
  tf.estimator.train_and_evaluate(
    estimator = estimator, 
    train_spec = train_spec, 
    eval_spec = eval_spec)
  
  return estimator

In [21]:
# Run the model
shutil.rmtree(arguments["output_dir"], ignore_errors = True) # start fresh each time
estimator = train_and_evaluate(arguments)

INFO:tensorflow:Using config: {'_model_dir': 'trained_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c935ba978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluat

## Get predictions

In [22]:
predictions_generator = estimator.predict(input_fn = read_dataset(filename = arguments['eval_file_pattern'], 
                                                                  mode = tf.estimator.ModeKeys.EVAL, 
                                                                  batch_size = arguments['batch_size'],
                                                                  params = arguments))
predictions_list = [x for x in predictions_generator]


read_dataset: _input_fn: filename = 
eval_data.csv
read_dataset: _input_fn: mode = 
eval
read_dataset: _input_fn: batch_size = 
16
read_dataset: _input_fn: params = 
{'train_file_pattern': 'train_data.csv', 'eval_file_pattern': 'eval_data.csv', 'output_dir': 'trained_model', 'batch_size': 16, 'reverse_feature_sequence': True, 'question_lstm_hidden_units': [256, 128, 64], 'question_lstm_dropout_output_keep_probs': [0.99, 0.97, 0.95], 'choices_lstm_hidden_units': [256, 128, 64], 'choices_lstm_dropout_output_keep_probs': [0.99, 0.97, 0.95], 'use_question_dnn': True, 'question_dnn_hidden_units': [64, 32, 16], 'question_dnn_dropout_rates': [0.01, 0.03, 0.05], 'use_choices_dnn': True, 'choices_dnn_hidden_units': [64, 32, 16], 'choices_dnn_dropout_rates': [0.01, 0.03, 0.05], 'final_logits_size': 16, 'train_steps': 1000, 'learning_rate': 0.1, 'start_delay_secs': 30, 'throttle_secs': 30}


read_dataset: _input_fn: file_list = 
['./eval_data.csv']
read_dataset: _input_fn: dataset.TextLineDatase

In [23]:
predictions_list[0]

{'cosine_similarities': array([-1.96403813e-34, -1.96403813e-34, -1.96403813e-34, -1.96403813e-34]),
 'euclidean_distances': array([2.80288289e-23, 2.80288289e-23, 2.80288289e-23, 2.80288289e-23]),
 'predicted_answer_index': 0,
 'question_text': b'<s> Which technology was developed most recently </s>',
 'predicted_answer_text': b'<s> cellular telephone </s>',
 'question_logits': array([ 3.43998133e-25, -3.02262859e-24, -1.78893753e-24, -2.49397701e-24,
        -1.73772735e-24,  6.54552521e-24, -1.97741558e-25,  5.74444957e-24,
        -4.16338954e-25,  1.53544742e-24, -1.99252766e-24,  2.73598812e-24,
         5.54520819e-24,  3.83728911e-24, -5.18451358e-24,  3.53377127e-24]),
 'choices_logits': array([[-3.43998133e-25,  3.02262859e-24,  1.78893753e-24,
          2.49397701e-24,  1.73772735e-24, -6.54552521e-24,
          1.97741558e-25, -5.74444957e-24,  4.16338954e-25,
         -1.53544742e-24,  1.99252766e-24, -2.73598812e-24,
         -5.54520819e-24, -3.83728911e-24,  5.18451358e